In [2]:
%%writefile assignment34.cu
// Задание 4
// Я взяла поэлементное умножение массива как программу для оптимизации
// Сравнить время выполнения для разных конфигураций блоков и сетки
#include <iostream>                 // Для работы с вводом/выводом
#include <cuda_runtime.h>           // Основные функции CUD
#include <chrono>                   // Для измерения времени выполнения

using namespace std;                // Чтобы не писать std::

// Поэлементное умножение
__global__ void multiply_kernel(float* arr, int n, float k)       // Kernel: каждый поток умножает один элемент
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;              // Глобальный индекс потока
    if (idx < n)                                                  // Проверка выхода за границы массива
    {
        arr[idx] *= k;                                            // Умножение элемента на константу
    }
}

int main()                                                        // Основная функция
{
    int n = 50000000;                                             // Размер массива
    size_t size = n * sizeof(float);                              // Размер массива в байтах
    float k = 2.5f;                                               // Константа умножение


    // Выделение и инициализация CPU массива
    float* h_arr = new float[n];                                  // Выделяем память в оперативной памяти
    for (int i = 0; i < n; i++)                                   // Заполняем массив начальными значениями
        h_arr[i] = 1.0f;                                          // Каждый элемент равен 1.0


    // Выделение GPU памяти
    float* d_arr;                                                 // Указатель на массив в памяти GPU
    cudaMalloc(&d_arr, size);                                     // Выделяем память в глобальной памяти GPU

    // Инициализация CUDA Events для точного замера времени
    cudaEvent_t start, stop;                                      // Создаём события начала и конца
    cudaEventCreate(&start);                                      // Инициализация события start
    cudaEventCreate(&stop);                                       // Инициализация события stop


    // Неоптимальная конфигурация блоков и сетки
    int blockSize_bad = 32;                                       // слишком маленький блок -> мало потоков в блоке
    int gridSize_bad = (n + blockSize_bad - 1) / blockSize_bad;   // Число блоков в сетке

    cudaMemcpy(d_arr, h_arr, size, cudaMemcpyHostToDevice);       // Копируем данные с CPU на GPU

    cudaEventRecord(start);                                       // Запускаем таймер
    multiply_kernel<<<gridSize_bad, blockSize_bad>>>(d_arr, n, k); // Запуск kernel
    cudaEventRecord(stop);                                        // Останавливаем таймер

    cudaEventSynchronize(stop);                                   // Ждём завершения kernel
    float elapsed_bad;                                            // Переменная для времени выполнения
    cudaEventElapsedTime(&elapsed_bad, start, stop);              // Получаем время в миллисекундах

    cout << "Время (Неоптимизировано): " << elapsed_bad << " мс" << endl; // Вывод результата

    // Оптимальная конфигурация блоков и сетки
    int blockSize_opt = 256;                                       // оптимальный размер блока
    int gridSize_opt = (n + blockSize_opt - 1) / blockSize_opt;   // Число блоков в сетке

    cudaMemcpy(d_arr, h_arr, size, cudaMemcpyHostToDevice);       // Копируем исходные данные

    cudaEventRecord(start);                                       // Запускаем таймер
    multiply_kernel<<<gridSize_opt, blockSize_opt>>>(d_arr, n, k); // Запуск kernel
    cudaEventRecord(stop);                                        // Останавливаем таймер

    cudaEventSynchronize(stop);                                   // Ждём завершения kernel
    float elapsed_opt;                                            // Переменная для времени выполнения
    cudaEventElapsedTime(&elapsed_opt, start, stop);              // Получаем время выполнения

    cout << "Время (Оптимизировано): " << elapsed_opt << " мс" << endl; // Вывод результата


    // Освобождение памяти
    cudaFree(d_arr);                                              // Освобождаем память на GPU
    delete[] h_arr;                                               // Освобождаем память на CPU
    cudaEventDestroy(start);                                      // Удаляем CUDA событие start
    cudaEventDestroy(stop);                                       // Удаляем CUDA событие stop

    return 0;                                                     // Завершения
}



Writing assignment34.cu


In [4]:
# Компиляция и запуск в Colab
!nvcc assignment34.cu -o assignment34
!./assignment34


Время (Неоптимизировано): 10.6699 мс
Время (Оптимизировано): 0.00288 мс
